In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import io
from scipy import stats

## Data Preprocessor

In [3]:
####DATA PREP####
from sklearn.preprocessing import Imputer, MinMaxScaler
def encode(data, cols):
    for i in cols:
        temp = pd.get_dummies(data[i], prefix=i)
        data = pd.concat([data, temp], axis=1)
        data.drop([data.keys()[-1],i], inplace = True, axis=1)
    return data
def fillnan(data,cols):
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    data[cols] =imp.fit_transform(data[cols])
    return data
def normalize_age(data,cols):
    scaler = MinMaxScaler()
    for i in cols:
        data[i] = scaler.fit_transform(data[i].values.reshape(-1,1))
    return data

In [4]:
####DATA CONVERSTION###
def split(d):
    X,y= d.iloc[:,1:],d.iloc[:,0]
    return X,y
def convert(X,y):
    X = X.values
    y = y.values[np.newaxis].T
    return X,y

In [20]:
###Processing DATA###
def Preprocess(Data,ttype):
    Data = Data.drop(["Name","Ticket","PassengerId","Cabin"],axis=1)
    if(ttype == "train"):
        X,y= split(Data)
        X = encode(X,['Sex','Embarked','SibSp','Parch','Pclass'])
        X = fillnan(X,['Age','Fare']) 
        X = normalize_age(X,['Age','Fare'])
        X,y = convert(X,y)
        return X,y
    else:
        X = Data
        X = encode(X,['Sex','Embarked','SibSp','Parch','Pclass'])
        X = fillnan(X,['Age','Fare']) 
        X = normalize_age(X,['Age','Fare'])
        X.drop(['Parch_6'],inplace = True,axis=1)
        X = X.values
        return X

In [17]:
d_tr = pd.read_csv("train.csv")
X_tr,y_tr = Preprocess(d_tr,"train")
print(X_tr.shape, y_tr.shape)

<class 'str'>
(891, 19) (891, 1)


In [11]:
d_tr.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [19]:
d_te = pd.read_csv("test.csv")
X_te = Preprocess(d_te,"test")
print(X_te.shape)

<class 'str'>
(418, 19)


In [ ]:
d_te.head

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tr, y_tr, test_size=0.2,train_size = 0.8, random_state=42)

## SKlearn Modules

In [10]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [17]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, np.squeeze(y_train))

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [22]:
acc = accuracy_score(y_test, clf.predict(X_test))
acc

0.8044692737430168

In [28]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=5, solver='lbfgs',multi_class='multinomial',n_jobs=2,max_iter=10000).fit(X_train, np.squeeze(y_train))
acc = accuracy_score(y_test, clf.predict(X_test))
acc

0.7988826815642458

In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=2000, random_state=0,max_features="sqrt",verbose=True).fit(X_train,y_train.ravel())
acc = accuracy_score(y_test, clf.predict(X_test))
acc

C:\Users\Vinu\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    0.1s finished


0.8100558659217877

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss
clf = SVC(kernel='rbf', max_iter=-1,gamma='auto',verbose=True)
clf.fit(X_train, y_train.ravel())
acc = accuracy_score(y_test, clf.predict(X_test))
acc

[LibSVM]

0.7821229050279329

In [14]:
from sklearn.metrics import r2_score
y_pred = clf.predict(X_test)
r2_score(y_test, y_pred)

[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    0.1s finished


0.2167310167310167